# Jigsaw - Agile Community Rules Classification
### https://www.kaggle.com/competitions/jigsaw-agile-community-rules

In [1]:
import kagglehub
import pandas as pd
import os

base_path = "./data/"
df_train = pd.read_csv(f"{base_path}train.csv")
print(df_train.shape)
df_train.head(1)

(2029, 9)


,row_id,body,rule,subreddit,positive_example_1,positive_example_2,negative_example_1,negative_example_2,rule_violation
0,0,Banks don't want you to know this! Click here ...,"No Advertising: Spam, referral links, unsolici...",Futurology,If you could tell your younger self something ...,hunt for lady for jack off in neighbourhood ht...,Watch Golden Globe Awards 2017 Live Online in ...,"DOUBLE CEE x BANDS EPPS - ""BIRDS""\n\nDOWNLOAD/...",0


In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
import torch
import os


# Paths
base_model_path = "unsloth/Qwen3-4B"  # Your base model
lora_path = "./lora/Qwen3_4B_lora_fp16_r64_s10000_e_3_msl2048/"             # Your LoRA checkpoint
merged_path = "./lora/Qwen3_4B_lora_fp16_r64_s10000_e_3_msl2048l_merged"               # Output for vLLM

# Load base model in FP16
model = AutoModelForCausalLM.from_pretrained(
    base_model_path,
    torch_dtype=torch.float16,
    device_map="auto"
)

# Load LoRA weights into base
model = PeftModel.from_pretrained(model, lora_path)

# Merge LoRA into the base weights
model = model.merge_and_unload()

# Save merged model
os.makedirs(merged_path, exist_ok=True)
model.save_pretrained(merged_path, safe_serialization=True)
tokenizer = AutoTokenizer.from_pretrained(base_model_path)
tokenizer.save_pretrained(merged_path)

print(f"Merged model saved to {merged_path}")



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.


Saving checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Merged model saved to ./lora/Qwen3_4B_lora_fp16_r64_s10000_e_3_msl2048l_merged
